In [ ]:
# pip install keybert
# pip install keybert transformers
# !pip install kiwipiepy

In [29]:
import pandas as pd
from keybert import KeyBERT
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel

# 데이터 Load & Preprocess

In [13]:
df=pd.read_csv('sample_split_sentences.csv',encoding='utf-8-sig')

In [14]:
df.head()

,Unnamed: 0,rst_name,review_text,review_sentence_split
0,0,가장맛있는족발 구의역점,잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서...,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...
1,1,강나루 유황오리주물럭 본점,생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ 늘 갈때마다 맛있어요 ㅎㅎ 가...,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ..."
2,2,고고기,금요일 저녁이라 고기가 땡겨서 고고기로 왔어요 고 기는 다 좋아하지만 두툼하고 질 ...,"['금요일 저녁이라 고기가 땡겨서 고고기로 왔어요', '고 기는 다 좋아하지만 두툼..."
3,3,고공 구의점,구의역 소고기 맛집입니다 토시살 갈비살 버 섯구이 구성의 소고기 세트 주문...,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세..."
4,4,고기로,고기로 돼지반판 주문했습니다 구의동 오션키즈 맞 은편이예요 옛 황금돼지 터 ...,"['고기로 돼지반판 주문했습니다', '구의동 오션키즈 맞 은편이예요', '옛 황금..."


In [6]:
df=df.drop(columns=['Unnamed: 0','review_text'])

In [ ]:
"""문자열 형태의 리스트를 파이썬 리스트로 변환하는 작업. 필요하면 사용"""
# import ast  # 문자열 형태의 리스트를 파이썬 리스트로 변환하기 위해 필요

# # 문자열 형태의 리스트를 파이썬 리스트로 변환
# df["review_sentence_split"] = df["review_sentence_split"].apply(ast.literal_eval)

# # 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인
# df["review_sentence_split"].iloc[0]

def text_join(row):
    text=row.review_sentence_split
    result=" ".join(text)
    return result

# df['joined_text']=df.apply(text_join,axis=1)
# df.drop(columns=['review_sentence_split'])

# nli-mean-tokens로 사전학습된 모델 가져옴

In [21]:
# 불필요하게 많은 공백은 처리해줌
document=df['review_text'].iloc[0]
document=document.replace('  ',' ')
document

'잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서 자주 봤던 곳인데 서비스 메뉴도 알차고 좋네요  매운 거 좋아  . 아이들이랑 가서 반반 족발 먹었어요 지점마다 서비스 내용이 다른가 봅니다 일단 콩나물국 정말 맛있었어요 서비스 5개 중에 막국수와 주먹밥 중에 엄청 고민했는데 아이들 위해서 주먹밥으로 결정했습니다 반찬들이 정갈하고 하나같이 맛이 좋아요 족발은 냄새 하나도 안나고 부드럽습니다 아이들은 기본 족발을 먹고 저는 매운 족발을 상추와 알배추에 싸 먹었어요 정말 맛있게 잘 먹었습니다 . 양도 푸짐하고 곁들여 먹을 수 있는 각종 김치랑 따뜻한 국물에 서비스 메뉴까지 나와서 좋아요. 구의동 먹자골목에서 롱런하는 이유가 있죠 맨날 배달로만 먹다가 매장에서 먹으니 더 꿀맛입니다 윤기가 좔좔 흐르는거 보이시죠 체인이라고 다 같은 체인은 아니잖아요 친절한 사장님 덕에 맛있고 편안하게 잘 먹고왔어요  매장 안 여기저기서 반반족발을 드시던데 진짜 둘 다 맛있어서 놓치고 싶지않아요 한톨도 남기지 않고 순삭하고는 터지는 배를 부여잡고 나왔네요 ㅎㅎ. 퇴근하고 현지인은 약속이라도 한듯 이 집으로 집결한다 왜이리 적게 받으시지 싶을정도로 음식에 진심인 집 오랫동안 텃세높은 광진구에서 살아남은 비결이 고스란히 느껴지는 집 1 보쌈 버너 먹는 동안 따뜻하고 촉촉한 고기를 맛 볼 수 있다 2 김치 보쌈 집의 근본인 김치가 무려 10종 김치를 먹기 위해 보쌈을 시킬 정도 3 서비스 따뜻한 콩나물국으로 입맛 돋우고 생굴 된찌 순두부찌개 막국수 주먹밥 중 취향껏 선택 4 푸짐한 양 둘이가서 푸파하려다가 실패하긴 처음 . 광진구족발원탑답게 대존맛임  역시 족발은 가족   넘넘 맛있게 먹었어요 서비스로 선택한 얼큰순두부찌개도 대존맛. 이름그대로 가장 맛있는 족발 보쌈 구의최고맛집입니다 믿고 드시면됨 서비스도 많음. 포장하러 갔는데 남자 사장님께서 너무 친절하시고 좋았어요  메뉴도 푸짐하고 양도 많고 전화로도 배민처럼 리뷰 이벤트 가능했는데 받고 오자마자 

In [23]:
kw_model = KeyBERT(model="distilbert-base-nli-mean-tokens")
keywords = kw_model.extract_keywords(document)
print(keywords)

[('광진구족발원탑답게', 0.9225), ('반반족발시켜', 0.8908), ('불친절합니다', 0.8865), ('잘먹구갑니당', 0.8823), ('날치알주먹밥을', 0.8792)]


# kiwi 사용 pos 태깅

In [ ]:
document

In [26]:
kiwi = Kiwi()
kiwi.analyze(document)

[([Token(form='잡내', tag='NNG', start=0, len=2),
   Token(form='안', tag='MAG', start=3, len=1),
   Token(form='나', tag='VV', start=4, len=1),
   Token(form='고', tag='EC', start=5, len=1),
   Token(form='맛있', tag='VA', start=7, len=2),
   Token(form='어요', tag='EC', start=9, len=2),
   Token(form='ᆼ', tag='Z_CODA', start=10, len=1),
   Token(form='구의', tag='NNP', start=12, len=2),
   Token(form='먹자골목', tag='NNG', start=15, len=4),
   Token(form='바로', tag='MAG', start=20, len=2),
   Token(form='앞', tag='NNG', start=23, len=1),
   Token(form='에서', tag='JKB', start=24, len=2),
   Token(form='접', tag='NNG', start=27, len=1),
   Token(form='근성', tag='NNG', start=29, len=2),
   Token(form='이', tag='JKS', start=31, len=1),
   Token(form='매우', tag='MAG', start=33, len=2),
   Token(form='좋', tag='VA', start=36, len=1),
   Token(form='습니다', tag='EC', start=37, len=3),
   Token(form='ᆼ', tag='Z_CODA', start=39, len=1),
   Token(form='지나가', tag='VV', start=41, len=3),
   Token(form='면서', tag='EC', st

# 특정 품사 구만 추출 하는 함수, 목적에 따라 바꿈

In [27]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith("N") or pos.startswith("SL") or pos.startswith("V"):
            results.append(token)
    return results

In [30]:
model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

In [31]:
def extract_keywords_from_reviews(document, top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1, 1),  # 단어 n-gram 범위
        stop_words=None,  # 불용어
        use_maxsum=False,
        use_mmr=True,
        diversity=0.2,  # 다양성
        top_n=top_n,
    )  # 상위 n개 키워드
    print(f"Keywords: {keywords}\n")
    print("-" * 80)  # 리뷰 사이 구분선

#### 현재는 명사,동사,형용사 구만 추출했는데, 전체 리뷰를 쓰되 부사구만 제거 해볼 예정

In [33]:
for i in range(3):
    #식당의 문장 리스트 추출
    review_doc = df["review_text"].iloc[i]
    review_noun_tagging=noun_extractor(review_doc)
    review_noun_tagging=" ".join(review_noun_tagging)

    print(f"Restaurant : {df['rst_name'].iloc[i]}")
    extract_keywords_from_reviews(review_noun_tagging)

Restaurant : 가장맛있는족발 구의역점
Keywords: [('좋아하', 0.7322), ('생각나', 0.7156), ('곁들이', 0.7109), ('지나가', 0.7086), ('맛나', 0.6955), ('쫄깃하', 0.6931), ('현지인', 0.6931), ('돋우', 0.6915), ('아이', 0.6876), ('밑반찬', 0.6802), ('쫀득하', 0.6794), ('정갈하', 0.679), ('원하', 0.671), ('드시', 0.6681), ('쏘맥', 0.6586), ('시키', 0.6576), ('말씀', 0.6545), ('문의', 0.6497), ('즐겁', 0.649), ('깨끗하', 0.63)]

--------------------------------------------------------------------------------
Restaurant : 강나루 유황오리주물럭 본점
Keywords: [('맛있', 0.6869), ('맛나', 0.646), ('배고프', 0.6431), ('반찬', 0.6221), ('맞이', 0.6221), ('훌륭하', 0.6164), ('한잔', 0.6161), ('어쩌', 0.6086), ('비하', 0.608), ('오리구', 0.606), ('식사', 0.6053), ('좋아하', 0.6046), ('편하', 0.603), ('비린내', 0.6029), ('왔습니', 0.5943), ('볶음밥', 0.5937), ('맛집', 0.5906), ('아저씨', 0.5884), ('챙기', 0.5809), ('더워요', 0.5788)]

--------------------------------------------------------------------------------
Restaurant : 고고기
Keywords: [('대존잘', 0.5886), ('셔보세요', 0.5837), ('못하', 0.5824), ('맛잏어요', 0.578), ('빵빵하', 0.5774

In [78]:
test = [
    "역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  .",
    "근처 순대국중 괜춘   .",
    "구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯.",
    "구의동 순대국 원탑맛집.",
    "오소리 국밥  순대국밥  야채순대 먹었어요",
    "2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   .",
    "고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요",
    "순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요",
    "와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ.",
    "어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당.",
    "순대국  고기모듬  술국 등 모든 메뉴가 맛있어요",
    "순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요",
    "순대국밥 이랑 막걸리 ㅋㅋ.",
    "순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃.",
    "순대국이 맛있어요.",
    "맛집이라 순대국 국물이 진해요",
    "순대국 깔끔.",
    "맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ.",
    "맛있는 순대국집  가격이 올라 별하나 뺍니다.",
    "순대국밥등 넘 맛있어요",
]

In [79]:
test1=" ".join(test)

In [80]:
test1

'역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  . 근처 순대국중 괜춘   . 구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯. 구의동 순대국 원탑맛집. 오소리 국밥  순대국밥  야채순대 먹었어요 2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   . 고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요 순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요 와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ. 어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당. 순대국  고기모듬  술국 등 모든 메뉴가 맛있어요 순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요 순대국밥 이랑 막걸리 ㅋㅋ. 순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃. 순대국이 맛있어요. 맛집이라 순대국 국물이 진해요 순대국 깔끔. 맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ. 맛있는 순대국집  가격이 올라 별하나 뺍니다. 순대국밥등 넘 맛있어요'

In [98]:
extract_keywords_from_reviews(test1)

Keywords: [('순대국밥을', 0.7202), ('순대국밥등', 0.6857), ('맛집이라', 0.6679), ('순대국밥', 0.641), ('끓여주셨어요', 0.6182), ('주문하면', 0.6155), ('잘먹었습니다', 0.6127), ('신경써주시면', 0.6055), ('적셔봅니당', 0.5961), ('원탑맛집', 0.5941), ('순대국중에', 0.5869), ('해줘서', 0.5843), ('맛은', 0.5792), ('데워달라니', 0.5782), ('시키면', 0.5751), ('괜춘', 0.5731), ('맛있고', 0.5728), ('맛집', 0.5698), ('부속고기들도', 0.5037), ('포장', 0.4673)]

--------------------------------------------------------------------------------


In [94]:
test2=noun_extractor(test1)

In [95]:
test2

['순대국',
 '먹거리',
 '맛있',
 '두',
 '둔하',
 '먹',
 '근처',
 '순대국',
 '구의역',
 '근처',
 '순대국',
 '일등',
 '맛집',
 '사장',
 '친절',
 '아쉽',
 '배민',
 '안함',
 '머릿고기',
 '순대국',
 '포장',
 '맛있',
 '위생',
 '신경',
 '쓰',
 '주',
 '좋',
 '구의동',
 '순대국',
 '맛집',
 '오소리',
 '국밥',
 '순대국',
 '야채',
 '순대',
 '먹',
 '포장',
 '주문',
 '대부분',
 '인분',
 '포장',
 '주',
 '하',
 '포장',
 '국물',
 '인분',
 '건더기',
 '인분',
 '하',
 '주',
 '포장',
 '아니',
 '이',
 '고기',
 '모듬',
 '시키',
 '순대',
 '수육',
 '부속',
 '고기',
 '나오',
 '좋',
 '순대국',
 '맛있',
 '소주',
 '마시',
 '식',
 '데우',
 '달',
 '끓이',
 '주',
 '구의',
 '먹',
 '보',
 '순대국',
 '이',
 '어르신',
 '사이',
 '끼',
 '막걸리',
 '순대국',
 '적시',
 '보',
 '순대국',
 '고기',
 '모듬',
 '술국',
 '메뉴',
 '맛있',
 '순대국',
 '내',
 '장국',
 '일대',
 '맛나',
 '중독',
 '가',
 '있',
 '순대국',
 '막걸리',
 '순대국',
 '얼큰하',
 '진하',
 '맛있',
 '모둠',
 '고기',
 '부드럽',
 '순대국',
 '맛있',
 '맛집',
 '이',
 '순대국',
 '국물',
 '진하',
 '순대국',
 '깔끔',
 '있',
 '모둠',
 '중양',
 '적',
 '아쉽',
 '맛있',
 '순대국',
 '가격',
 '오르',
 '하나',
 '빼',
 '순대',
 '국밥',
 '넘',
 '맛있']

In [96]:
" ".join(test2)

'순대국 먹거리 맛있 두 둔하 먹 근처 순대국 구의역 근처 순대국 일등 맛집 사장 친절 아쉽 배민 안함 머릿고기 순대국 포장 맛있 위생 신경 쓰 주 좋 구의동 순대국 맛집 오소리 국밥 순대국 야채 순대 먹 포장 주문 대부분 인분 포장 주 하 포장 국물 인분 건더기 인분 하 주 포장 아니 이 고기 모듬 시키 순대 수육 부속 고기 나오 좋 순대국 맛있 소주 마시 식 데우 달 끓이 주 구의 먹 보 순대국 이 어르신 사이 끼 막걸리 순대국 적시 보 순대국 고기 모듬 술국 메뉴 맛있 순대국 내 장국 일대 맛나 중독 가 있 순대국 막걸리 순대국 얼큰하 진하 맛있 모둠 고기 부드럽 순대국 맛있 맛집 이 순대국 국물 진하 순대국 깔끔 있 모둠 중양 적 아쉽 맛있 순대국 가격 오르 하나 빼 순대 국밥 넘 맛있'

In [97]:
extract_keywords_from_reviews(" ".join(test2))

Keywords: [('맛나', 0.6302), ('건더기', 0.6231), ('진하', 0.6197), ('술국', 0.6197), ('구의역', 0.6119), ('시키', 0.6022), ('둔하', 0.5901), ('구의', 0.5859), ('근처', 0.5839), ('데우', 0.5835), ('안함', 0.5728), ('막걸리', 0.564), ('오소리', 0.5308), ('인분', 0.5233), ('주문', 0.5018), ('순대', 0.4948), ('소주', 0.4714), ('적시', 0.4641), ('하나', 0.4565), ('머릿고기', 0.4538)]

--------------------------------------------------------------------------------
